## 라이브러리 로드

In [ ]:
# 데이터 분석을 위한 pandas, 수치계산을 위한 numpy, 시각화를 위한 seaborn, matplotlib 을 로드합니다.
import pandas as pd
import numpy as np

## 데이터 로드

In [ ]:
# 데이콘의 해당 데이터셋은 CC-BY-4.0 라이센스입니다.
# 데이터 출처 : https://dacon.io/competitions/official/235747/data
# 로컬 PC에서 실습 시 직접 데이콘 사이트에 회원가입하고 다운로드 해주세요.

import os, platform
base_path = "data/klue/"

def file_exist_check(base_path):
    if os.path.exists(f"{base_path}train_data.csv"):
        print(f"{base_path} 경로에 파일이 이미 있음")
        return
    
    if platform.system() == "Linux":
        print(f"파일을 다운로드 하고 {base_path} 경로에 압축을 해제함")
        !wget https://bit.ly/dacon-klue-open-zip
        if not os.path.exists(base_path):
            os.makedirs(base_path)
        !unzip dacon-klue-open-zip -d data/klue
    else:
        print(f"""https://dacon.io/competitions/official/235747/data 에서 다운로드 하고
              실습 경로 {base_path}에 옮겨주세요.""")
    return
    
file_exist_check(base_path) 

In [ ]:
# 학습, 예측 데이터셋을 불러옵니다.
train = pd.read_csv(f"{base_path}train_data.csv")
test = pd.read_csv(f"{base_path}test_data.csv")
train.shape, test.shape

In [ ]:
# 토픽을 불러옵니다.
topic = pd.read_csv(f"{base_path}topic_dict.csv")
topic

## 문자 전처리

In [ ]:
# 정규표현식
import re

def preprocessing(text):
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 중복으로 생성된 공백값을 제거합니다.
    text = re.sub('[\s]+', ' ', text)
    # 영문자를 소문자로 만듭니다.
    text = text.lower()
    return text

In [ ]:
# map을 통해 전처리 일괄 적용
train["title"] = train["title"].map(preprocessing)
test["title"] = test["title"].map(preprocessing)

## 형태소 분석
* [KoNLPy: 파이썬 한국어 NLP — KoNLPy documentation](https://konlpy.org/ko/latest/)
    * Hannanum Class
    * Kkma Class
    * Komoran Class
    * Mecab Class
    * Okt Class

```sh
pip install --upgrade pip
pip install JPype1
pip install konlpy --upgrade
```

다음의 항목을 만족해야 konlpy를 윈도우나 맥에서 사용할 수 있습니다.
대부분의 오류는 환경변수나 최신버전의 JDK가 설치되지 않아서 입니다. 공식문서에서는 1.7 이상의 JDK 를 권장하고 있습니다.

```
1) 최신 버전의 JAVA(JDK)를 설치
2) JAVA_HOME 환경변수를 추가
3) path 환경변수에 %JAVA_HOME%\bin; 추가
```

설치 관련 자세한 내용은 다음을 참고해 보세요.

https://konlpy.org/ko/latest/install/

In [ ]:
# konlpy 가 설치되어 있지 않다면 설치합니다. 
# konlpy 는 다른 프로그래밍 언어(JAVA, C++)로 만들어진 형태소 분석기를 파이썬 인터페이스로 사용할 수 있는 도구 입니다.
# JPype1도 파이썬에서 자바를 사용할 수 있도록 하는 도구입니다.
# 인터페이스가 파이썬이지만 내부는 해당 언어로 동작하여 다른 언어도 함께 설치되어 있어야 합니다.
# 그래서 설치는 꼭 공식문서를 참고해서 합니다.
# Google Colab 에서는 아래 pip 구문 만으로 설치할 수 있으나 다른 운영체제 사용시 설치 문서를 참고해 주세요!
# !pip install konlpy --upgrade

In [ ]:
small_text = "버스의 운행시간을 문의합니다. 어?!"
small_text

In [ ]:
# Kkma


In [ ]:
# # Colab에 Mecab 설치
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# %cd Mecab-ko-for-Google-Colab
# !bash install_mecab-ko_on_colab190912.sh


In [ ]:
# Mecab(윈도우는 지원하지 않음, colab에서도 별도의 설치가 필요)


### 어간 추출(語幹 抽出, 영어: stemming)

형태론 및 정보 검색 분야에서 어형이 변형된 단어로부터 접사 등을 제거하고 그 단어의 어간을 분리해 내는 것을 의미한다. 여기서 어간은 반드시 어근과 같아야 할 필요는 없으며, 어근과 차이가 있더라도 관련이 있는 단어들이 일정하게 동일한 어간으로 맵핑되게 하는 것이 어간 추출의 목적이다. 1960년대부터 컴퓨터 과학 분야에서 다양한 어간 추출 관련 알고리즘들이 연구되어 왔다. 많은 웹 검색 엔진들은 동일한 어간을 가진 단어들을 동의어로 취급하는 방식으로 질의어 확장을 하여 검색 결과의 품질을 높인다.

어간 추출 프로그램은 흔히 스테밍 알고리즘(stemming algorithm) 또는 스테머(stemmer)라 불린다.

* [어간 추출 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EC%96%B4%EA%B0%84_%EC%B6%94%EC%B6%9C)

In [ ]:
# Okt
# steming 기능을 제공


In [ ]:
# 형태소 분석기(Okt) 불러오기 
# ['Josa', 'Eomi', 'Punctuation'] 조사, 어미, 구두점 제거

# def okt_clean(text):
#     clean_text = []

In [ ]:
# !pip install tqdm --upgrade

In [ ]:
# tqdm 으로 전처리 진행 상태를 표시
from tqdm import tqdm
tqdm.pandas() 

train['title']
test['title']

## 불용어 제거

In [ ]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = [ '합니다', '하는', '할', '하고', '한다', 
             '그리고', '입니다', '그', '등', '이런', '및','제', '더']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [ ]:
train["title"] = train["title"].map(remove_stopwords)
test["title"] = test["title"].map(remove_stopwords)
train["title"]

## 학습, 예측 데이터셋 만들기

In [ ]:
# 학습과 훈련에 사용 할 기출문제와 실전 예측에 사용할 실전문제를 만듭니다.
# X_train_text
# X_test_text

In [ ]:
# 정답값으로 사용할 topic_idx 를 변수에 담아 재사용 합니다.
# label_name

In [ ]:
# 기출문제의 정답을 만들어 줍니다.
# 잘 나뉘었는지 정닶 값의 빈도수를 확인해 봅니다.
# y_train

## 벡터화
* 머신러닝이나 딥러닝 알고리즘은 문자를 이해할 수 없습니다. 내부에서는 수치 계산이 이루어지기 때문에 문자를 숫자로 변경해 주어야 합니다.


### TF-IDF(Term Frequency - Inverse Document Frequency)

In [ ]:
# TfidfVectorizer 로 벡터화 합니다.
# fit 으로 변환할 어휘를 학습합니다.
# tfidfvect


In [ ]:
# transform
# X_train
# X_test

## 모델


전체 과정
* feature_names : 학습(훈련), 예측에 사용할 단어 사전 입니다.
* label_name : 정답값

* X_train : feature_names 에 해당되는 컬럼만 train에서 가져옵니다.
    * 학습(훈련)에 사용할 데이터셋 예) 시험의 기출문제

* X_test : feature_names 에 해당되는 컬럼만 test에서 가져옵니다.
    * 예측에 사용할 데이터셋 예) 실전 시험문제

* y_train : label_name 에 해당 되는 컬럼만 train에서 가져옵니다.
    * 학습(훈련)에 사용할 정답 값 예) 기출문제의 정답

* model : 학습, 예측에 사용할 머신러닝 알고리즘

* model.fit(X_train, y_train) : 학습(훈련), 기출문제와 정답을 가지고 학습(훈련)하는 과정과 유사합니다.

* model.predict(X_test) :  예측, 실제 시험을 보는 과정과 유사합니다. => 문제를 풀어서 정답을 구합니다.

* score
    * 시험을 봤다면 몇 문제를 맞고 틀렸는지 채점해 봅니다.
* metric
    * 점수를 채점하는 공식입니다. (예를 들어 학교에서 중간고사를 봤다면 전체 평균을 계산해 줍니다.)
    
    
### DecisionTree와 Randomforest의 Parameter
* n_estimators
    * DecisionTree 는 트리를 하나만 만들기 때문에 없음
    * 결정트리의 개수, 많을 수록 성능이 좋아지나 오래걸리며 샘플의 수에 따라 트리를 만들 수 있는 개수가 제한적
* max_depth
    * 트리의 최대 깊이
* max_features	
    * 피처의 개수(int) 혹은 비율(float)
    * sqrt 또는 auto : 전체 피처 중 √(피처개수) 만큼 사용
    * log : 전체 피처 중 log2(전체 피처 개수) 만큼 
* min_samples_split
    * 노드를 분할하기 위한 최소 샘플 데이터수
* min_samples_leaf
    * 리프노드가 되기 위해 필요한 최소 샘플 데이터수
* max_leaf_nodes
    * 리프노드의 최대 개수
* n_jobs
    * 사용할 CPU 코어의 수, -1로 설정 시 사용할 수 있는 모든 CPU코어를 사용
    
## 학습과 예측

In [ ]:
# DecisionTreeClassifier


## Parameter Search
<img src="https://i.imgur.com/h8QtsMz.png" width=500>

### Grid Search

In [ ]:
# GridSearchCV
# gscv

In [ ]:
# best_estimator_


In [ ]:
# best_score_


### Random Search

In [ ]:
# RandomizedSearchCV


In [ ]:
# best_estimator_


In [ ]:
# best_score_


## 예측

In [ ]:
# 실전 시험과 유사합니다. 정답 없이 문제만 넣고 풀어서 예측합니다.
# predict로 예측합니다. 
# y_predict

## 정답값 로드

In [ ]:
# sample_submission 파일을 불러옵니다.
# Dacon 에 제출할 답안지 양식입니다.
submit = pd.read_csv(f"{base_path}sample_submission.csv")
submit.head()

In [ ]:
# 정답값 측정을 위해 y_test 변수에 할당
# submit["topic_idx"]

In [ ]:
file_name = f"{base_path}submit_rscv_{rscv.best_score_}.csv"
file_name

In [ ]:
# 잘 저장이 되었는지 확인합니다.


In [ ]:
# 잘 저장이 되었는지 확인합니다.


* dacon에 제출해서 리더보드 확인하기 : https://dacon.io/competitions/official/235747/mysubmission


## 여러 방법을 통해 예측비율을 높여보세요. 
* 아래 항목 외의 기법을 사용해도 됩니다.
 * 전처리 하기
 * 불용어 처리
 * BOW, TF-IDF의 파라메터 변경
 * 분류기의 파라메터 변경
 * 분류기 변경